# Projeto 1 Visão Computacional 2020
## Gabriel Zezze & Pedro Luiz

Importando pacotes

In [107]:
import G6_iris_recognition
import cv2
import os
import numpy as np
import shutil


Montando o google drive localmente para ter acesso as imagens

In [179]:
RAW_IMAGES_PATH = '../assets/raw_images'
SHARPENED_IMAGES_PATH = './sharpened_images'
TEST_IMAGES_PATH = './test_images'

shutil.rmtree(SHARPENED_IMAGES_PATH, ignore_errors=True)
shutil.rmtree(TEST_IMAGES_PATH, ignore_errors=True)

if not os.path.exists(SHARPENED_IMAGES_PATH):
  os.mkdir(SHARPENED_IMAGES_PATH)

if not os.path.exists(TEST_IMAGES_PATH):
  os.mkdir(TEST_IMAGES_PATH)

Blacklist de imagens invalidas de treinamentos prévios

In [180]:
blacklist = [
    "0001/0001_005.bmp",
    "0001/0001_004.bmp",
    "0001/0001_011.bmp",
    "0001/0001_006.bmp",
    "0001/0001_019.bmp",
    "0002/0002_014.bmp",
    "0002/0002_015.bmp",
    "0002/0002_016.bmp",
    "0002/0002_013.bmp",
    "0002/0002_017.bmp",
    "0000/0000_016.bmp",
    "0000/0000_008.bmp",
    "0000/0000_004.bmp",
    "0003/0003_018.bmp",
    "0003/0003_016.bmp",
    "0003/0003_017.bmp",
    "0003/0003_005.bmp",
    "0003/0003_013.bmp",
    "0003/0003_004.bmp",
    "0003/0003_012.bmp",
    "0006/0006_009.bmp",
    "0007/0007_007.bmp",
    "0007/0007_019.bmp",
    "0007/0007_001.bmp",
    "0008/0008_015.bmp",
    "0008/0008_010.bmp",
    "0008/0008_019.bmp",
    "0008/0008_006.bmp",
    "0011/0011_000.bmp",
    "0011/0011_013.bmp",
    "0016/0016_005.bmp",
    "0016/0016_011.bmp",
    "0016/0016_001.bmp",
    "0016/0016_006.bmp",
    "0017/0017_016.bmp",
    "0017/0017_010.bmp",
    "0017/0017_008.bmp",
    "0018/0018_013.bmp",
    "0018/0018_009.bmp",
    "0018/0018_006.bmp",
    "0018/0018_015.bmp",
    "0019/0019_014.bmp",
    "0019/0019_003.bmp",
    "0019/0019_000.bmp",
    "0019/0019_008.bmp",
    "0020/0020_005.bmp",
    "0020/0020_017.bmp",
    "0020/0020_009.bmp",
    "0020/0020_007.bmp",
    "0021/0021_008.bmp",
    "0021/0021_017.bmp",
    "0021/0021_003.bmp",
    "0021/0021_002.bmp",
    "0026/0026_008.bmp",
    "0026/0026_004.bmp",
    "0027/0027_013.bmp",
    "0027/0027_012.bmp",
    "0030/0030_013.bmp",
    "0030/0030_002.bmp",
    "0029/0029_001.bmp",
    "0029/0029_018.bmp",
    "0031/0031_002.bmp",
    "0031/0031_003.bmp",
    "0031/0031_015.bmp",
    "0027/0027_013.bmp",
    "0027/0027_019.bmp"
]

folder_blacklist = ["0005", "0006", "0009", "0010"]

Utilities Functions

In [181]:
def equalize_img(img):
  return cv2.equalizeHist(img)

def crop_image(img, cordenadas1, cordenadas2):
  x1, y1 = cordenadas1
  x2, y2 = cordenadas2

  h, w = img.shape
  return img[y1 : h - y2][x1 : w - x2]

In [182]:
cont2 = 0
for folder in ["0000", "0001", "0002", "0003", "0004",
               "0005", "0006", "0007", "0008", "0009", 
               "0010", "0011", "0016", "0017", "0018",
               "0019", "0020", "0021", "0026"]:
  if folder in folder_blacklist:
    continue
  cont2 = 0
  if folder == '.ipynb_checkpoints' or folder == '.DS_Store':
    continue
  if not os.path.exists(f'{SHARPENED_IMAGES_PATH}/{folder}'):
    os.mkdir(f'{SHARPENED_IMAGES_PATH}/{folder}')
  if not os.path.exists(f'{TEST_IMAGES_PATH}/{folder}'):
    os.mkdir(f'{TEST_IMAGES_PATH}/{folder}')
  for file in os.listdir(f'{RAW_IMAGES_PATH}/{folder}'):
    if f"{folder}/{file}" in blacklist:
      continue

    img = cv2.cvtColor(cv2.imread(f'{RAW_IMAGES_PATH}/{folder}/{file}'), cv2.COLOR_BGR2GRAY)
    h, w = img.shape

    img_cropped = img
    y2 = img_cropped.shape[0]
    y1 = 0

    initial_crop = (20, 35)
    crop_cord = initial_crop

    while abs(y2 - y1) > img_cropped.shape[0] * 0.70 and not img_cropped.shape[0] < img.shape[1] * 0.5:

      crop_cord = (initial_crop[0], crop_cord[1] + 2)
      crop_cord2 = initial_crop
      img_cropped = crop_image(img_cropped, crop_cord, crop_cord2)

      img_equalized = equalize_img(img_cropped)
      img_blur = cv2.medianBlur(img_equalized, 25)

      img_border_detected = cv2.Canny(img_blur, 50, 100)

      pts = np.argwhere(img_border_detected > 0)
      y1,x1 = pts.min(axis=0)
      y2,x2 = pts.max(axis=0)

    blank_image = np.zeros((h, w), np.uint8)
    final_crop = img_equalized[y1 : y2 , x1 : x2]

    blank_image[ y1 : y2, x1 : x2 ] = final_crop

    final_crop = blank_image

    img_equalized = cv2.cvtColor(img_equalized, cv2.COLOR_GRAY2BGR)
    rec = cv2.rectangle(img_equalized, (x1,y1), (x2,y2), (0,0,255), 3, cv2.LINE_AA)

    if cont2 < 8:
      cv2.imwrite(f'{SHARPENED_IMAGES_PATH}/{folder}/{file}', final_crop)
    else:
      cv2.imwrite(f'{TEST_IMAGES_PATH}/{folder}/{file}', final_crop)
    cont2 += 1

In [183]:
G6_iris_recognition.iris_model_train(SHARPENED_IMAGES_PATH, "model.pickle")

directory_list ['./sharpened_images/0018', './sharpened_images/0020', './sharpened_images/0001', './sharpened_images/0016', './sharpened_images/0011', './sharpened_images/0002', './sharpened_images/0008', './sharpened_images/0000', './sharpened_images/0019', './sharpened_images/0021', './sharpened_images/0026', './sharpened_images/0007', './sharpened_images/0003', './sharpened_images/0017', './sharpened_images/0004']
name 0018
path_to_image ./sharpened_images/0018/0018_012.bmp
path_to_image ./sharpened_images/0018/0018_003.bmp
path_to_image ./sharpened_images/0018/0018_004.bmp
path_to_image ./sharpened_images/0018/0018_014.bmp
path_to_image ./sharpened_images/0018/0018_005.bmp
path_to_image ./sharpened_images/0018/0018_002.bmp
path_to_image ./sharpened_images/0018/0018_008.bmp
path_to_image ./sharpened_images/0018/0018_001.bmp
name 0020
path_to_image ./sharpened_images/0020/0020_008.bmp
path_to_image ./sharpened_images/0020/0020_006.bmp
path_to_image ./sharpened_images/0020/0020_001.bm

['0018',
 '0020',
 '0001',
 '0016',
 '0011',
 '0002',
 '0008',
 '0000',
 '0019',
 '0021',
 '0026',
 '0007',
 '0003',
 '0017',
 '0004']

In [184]:
hit = 0
miss = 0

for folder in os.listdir(TEST_IMAGES_PATH):
  if folder == '.ipynb_checkpoints' or folder == '.DS_Store':
    continue
    
  for file in os.listdir(f'{TEST_IMAGES_PATH}/{folder}'):
    if f"{folder}/{file}" in blacklist:
      continue
    iris_name = G6_iris_recognition.iris_model_test("model.pickle", f'{TEST_IMAGES_PATH}/{folder}/{file}')
    print(iris_name, folder)
    print("---------- \n")
    if iris_name == folder:
      hit += 1 
    else:
      miss += 1
print("Acertos:", hit)
print("Erros: ", miss)
print("Porcentagem de acertos: ", hit/(hit + miss))

0000 0008
---------- 

0004 0008
---------- 

0004 0008
---------- 

0004 0008
---------- 

0017 0008
---------- 

0000 0008
---------- 

0004 0008
---------- 

0008 0008
---------- 

0017 0002
---------- 

0004 0002
---------- 

0004 0002
---------- 

0004 0002
---------- 

0004 0002
---------- 

0000 0002
---------- 

0004 0002
---------- 

0011 0011
---------- 

0011 0011
---------- 

0011 0011
---------- 

0011 0011
---------- 

0011 0011
---------- 

0004 0011
---------- 

0011 0011
---------- 

0011 0011
---------- 

0011 0011
---------- 

0011 0011
---------- 

0004 0016
---------- 

0004 0016
---------- 

0004 0016
---------- 

0004 0016
---------- 

0017 0016
---------- 

0004 0016
---------- 

0018 0016
---------- 

0004 0016
---------- 

0001 0001
---------- 

0011 0001
---------- 

0011 0001
---------- 

0001 0001
---------- 

0011 0001
---------- 

0011 0001
---------- 

0007 0001
---------- 

0004 0018
---------- 

0004 0018
---------- 

0004 0018
---------- 

0007 0018
-